   # Python FHIR Demo: Saving data to a server

In [1]:
### Install fhirclient module using sys.executable (the current python)
#import sys
#!{sys.executable} -m pip install --user jupyter fhirclient

In [2]:
### Import required modules / models
import json
from fhirclient import client
import fhirclient.server as s
import fhirclient.models.patient as p
import fhirclient.models.humanname as hn
import fhirclient.models.medicationstatement as ms
from IPython.core.display import display, HTML, JSON

In [3]:
### Define a few functions to display output nicely
def pretty(js):
    return json.dumps(js, indent=2)
def prettyprint(x):
    print(pretty(x))

In [4]:
### Set a few options to pass into the FHIRClient object instantiator
settings = {
    'app_id': 'my_web_app',
    #'api_base': 'https://vonk.fire.ly/'	
    'api_base': 'https://r3.smarthealthit.org/'
}
### Create an instanct of the fhirclient engine
smart = client.FHIRClient(settings=settings)
### This will do a pre-check to see if any auth is necessary (it's not for this one, so expect 'True')
smart.prepare()
myserver = s.FHIRServer(client, base_uri="https://r3.smarthealthit.org/")

In [12]:
pat = p.Patient()
name = hn.HumanName()
name.given = ["Samuel"]
name.family = "Fitchbasket" # <-- put a new last name here!
pat.name = [name]
myserver.post_json("/Patient",pat.as_json()) 

<Response [201]>

In [19]:
search = p.Patient.where({'name':'Fitchbasket'}) # <-- search for your new last name here!
patients = search.perform_resources(smart.server)
for patient in patients:
    prettyprint(patient.as_json())


{
  "id": "235262",
  "meta": {
    "lastUpdated": "2020-11-11T10:56:39.579-05:00",
    "versionId": "1"
  },
  "name": [
    {
      "family": "Fitchbasket",
      "given": [
        "Samuel"
      ]
    }
  ],
  "resourceType": "Patient"
}


In [18]:
patient = p.Patient.read('235261', smart.server)
prettyprint(patient.as_json())

{
  "id": "235261",
  "meta": {
    "lastUpdated": "2020-11-11T10:58:21.897-05:00",
    "versionId": "2"
  },
  "name": [
    {
      "family": "Jones",
      "given": [
        "Harold"
      ]
    }
  ],
  "resourceType": "Patient"
}


In [17]:
patient.name[0].family = "Jones"
myserver.put_json("/Patient/235261"
                  , patient.as_json())

<Response [200]>

In [ ]:
myserver.delete_json("/Patient/235258") # <-- replace with an ID from above